## Adding Labels to Your Variables

In [ ]:
import saspy
sas = saspy.SASsession()

In [ ]:
%%SAS
libname Learn 'C:/Users/vignesh/SAS/learningsas/misc/test1';

* Labels character limit - 256 characters long

In [ ]:
%%SAS
data Learn.Test_Scores;
   length ID $ 3 Name $ 15;
   input ID $ Score1-Score3 Name $;
   label ID='Student ID' 
     Score1='Math Score' 
     Score2='Science Score' 
     Score3='English Score';
   datalines;
1 90 95 98 Jan
2 78 77 99 Preston
3 88 91 92 Russell
;
run;


proc means data=Learn.Test_Scores;
run;

### Using PROC FORMAT to Create User-Defined Formats

In [ ]:
%%SAS
proc format;
   value $Gender 'M' = 'Male' 
                 'F' = 'Female' 
                 ' ' = 'Not entered' 
               other = 'Miscoded';
               
   value Age low-29 = 'Less than 30' 
              30-50 = '30 to 50' 
            51-high = '51+';
            
   value $Likert '1' = 'Strongly Disagree' 
                 '2' = 'Disagree' 
                 '3' = 'No Opinion' 
                 '4' = 'Agree' 
                 '5' = 'Strongly Agree';
run;


- Expection - format names cannot end in a digit and the maximum length of a format name is 31 characters

- The keyword `other` in the VALUE statement causes the text `Miscoded` to be printed for any characters besides `M`, `F`, or a missing value.

- In the AGE format, the keywords LOW and HIGH refer to the lowest nonmissing value and the highest value, respectively.

- The last format, $LIKERT, is used to substitute the appropriate text for the digits _1 (strongly disagree)_ to _5 (strongly agree)_.

In [ ]:
%%SAS
data learn.survey;
   infile 'C:/Users/vignesh/SAS/learningsas/data/survey.txt' pad;
   input ID : $3.
         Gender : $1.
         Age
         Salary
         (Ques1-Ques5)(: $1.);
run;

proc print data=Learn.Survey;
   id ID;
   var Gender Age Salary Ques1-Ques5;
   format Gender      $Gender.
          Age         Age.
          Ques1-Ques5 $Likert.
          Salary      Dollar11.2;
run;

### Regrouping Values Using a Format

In [ ]:
%%SAS
proc format;
   value $Three 1, 2='Disagreement' 
                   3='No opinion' 
                4, 5='Agreement';
run;

### Applying the New Format to Several Variables with PROC FREQ 

In [ ]:
%%SAS
proc freq data=Learn.Survey;
   tables Ques1-Ques5;
   format Ques1-Ques5 $Three.;
run;

### More on Format Ranges

```
value $Gradefmt 'A' – 'C' = 'Passing'
                      'D' = 'Borderline'
                      'F' = 'Failing'
                  'I','W' = 'Incomplete or withdrew'
                      ' ' = 'Not recorded'
                    other = 'Miscoded';
```
```
value Age low-29 = 'Less than 30'
           30-50 = '30 to 50'
         51-high = '51+';
```
```
value Age low-<30 = 'Less than 30'
           30-<51 = '30 to less than 51'
          51-high = '51+';
```
```
value Age low-30 = 'Less than or equal to 30'
          30<-51 = 'Greater than 30 to 51'
        51<-high = 'Greater than 51';
```

### Storing Your Formats in a Format Library

In [ ]:
%%SAS
libname Myfmts 'C:/Users/vignesh/SAS/learningsas/misc/test1';

proc format library=Myfmts;
   value $Gender 'M' = 'Male'
                 'F' = 'Female'
                 ' ' = 'Not entered'
               other = 'Miscoded';
               
   value Age low-29  = 'Less than 30'
             30-50   = '30 to 50'
             51-high = '51+';
             
   value $Likert '1' = 'Strongly disagree'
                 '2' = 'Disagree'
                 '3' = 'No opinion'
                 '4' = 'Agree'
                 '5' = 'Strongly agree';
run;

```
options fmtsearch=(Myfmts);
options fmtsearch=(myfmts Work Library);
```

### Adding LABEL and FORMAT Statements in the DATA Step

In [ ]:
%%SAS
libname Learn 'C:/Users/vignesh/SAS/learningsas/misc/test1';
libname Myfmts 'C:/Users/vignesh/SAS/learningsas/misc/test1';

options fmtsearch=(Myfmts);

data Learn.Survey;
   infile 'C:/Users/vignesh/SAS/learningsas/data/Survey.txt' pad;
   input ID : $3.
         Gender : $1.
         Age
         Salary
         (Ques1-Ques5)(: $1.);
   format Gender      $Gender.
          Age         Age.
          Ques1-Ques5 $Likert.
          Salary      Dollar10.0;
   label ID     = 'Subject ID'
         Gender = 'Gender'
         Age    = 'Age as of 1/1/2006'
         Salary = 'Yearly Salary'
         Ques1  = 'The governor is doing a good job?'
         Ques2  = 'The property tax should be lowered'
         Ques3  = 'Guns should be banned'
         Ques4  = 'Expand the Green Acre program'
         Ques5  = 'The school needs to be expanded';
run;

### Running PROC CONTENTS on a Data Set with Labels and Formats

In [ ]:
%%SAS
proc contents data=Learn.Survey varnum;
run;

### Displaying Format Definitions in a User-created Library

In [ ]:
%%SAS
title "Format Definitions in the Myfmts Library";
proc format library=Myfmts fmtlib;
run;

### Demonstrating a SELECT Statement with PROC FORMAT

In [ ]:
%%SAS
proc format library=Myfmts;
   select Age $Likert;
run;